# MPC Orbits API

#### This tutorial provides information on how to use the Minor Planet Center's Orbits API.

The Minor Planet Center's `Orbits` service returns orbital elements and related parameters for solar system objects.

This is useful when you want to:
 - Retrieve the best-fit orbital elements for an asteroid or comet
 - Access orbital uncertainties and covariance information
 - Get orbit-derived quantities like absolute magnitude, epoch, and orbital classification
 - Perform ephemeris calculations or orbit propagation

The Orbits API is a REST endpoint. You can send GET requests to:

    https://data.minorplanetcenter.net/api/get-orb

In the examples below we use Python code to query the API.

Further information and documentation can be found at:
 - https://minorplanetcenter.net/mpcops/documentation/orbits-api/
 - https://github.com/Smithsonian/mpc-public/tree/main/mpc_orb (Python package for working with mpc_orb format)

# Import Packages
Here we import the standard Python packages needed to call the API and interpret the returned data.

In [ ]:
import requests
import json

# API Parameters

The Orbits API accepts the following parameters:

| Parameter | Type | Required | Description |
|-----------|------|----------|-------------|
| `desig` | String | Yes | Name, permanent number, or provisional designation of the object |

**Note:** Both packed and unpacked designation formats are supported. Currently limited to single object queries.

### Response Format

The API returns data in the `mpc_orb` JSON format, which contains comprehensive orbital information including:
- Keplerian orbital elements (a, e, i, node, argperi, meananomaly)
- Epoch and coordinate system information
- Physical parameters (H, G)
- Orbital uncertainties and covariance matrix
- Object classification and identifiers

# Basic Query: By Name

Here we query for the orbital elements of asteroid `Ceres`, the first discovered asteroid.

In [ ]:
# Query orbit for Ceres by name
response = requests.get(
    "https://data.minorplanetcenter.net/api/get-orb",
    json={"desig": "Ceres"}
)

if response.ok:
    result = response.json()
    mpc_orb = result[0]['mpc_orb']
    print(json.dumps(mpc_orb, indent=4))
else:
    print(f"Error: {response.status_code}")
    print(response.content)

# Query by Permanent Number

You can also query using the asteroid's permanent number.

In [ ]:
# Query orbit by permanent number
response = requests.get(
    "https://data.minorplanetcenter.net/api/get-orb",
    json={"desig": "433"}  # 433 Eros
)

if response.ok:
    mpc_orb = response.json()[0]['mpc_orb']
    print(f"Object: {mpc_orb.get('designation_data', {}).get('iau_name', 'Unknown')}")
    print(f"Number: {mpc_orb.get('designation_data', {}).get('number')}")
else:
    print(f"Error: {response.status_code}")

# Query by Provisional Designation

Provisional designations work too, in either packed or unpacked format.

In [ ]:
# Query by unpacked provisional designation
response = requests.get(
    "https://data.minorplanetcenter.net/api/get-orb",
    json={"desig": "2023 BU"}  # Famous close-approach asteroid
)

if response.ok:
    mpc_orb = response.json()[0]['mpc_orb']
    desig_data = mpc_orb.get('designation_data', {})
    print(f"Primary designation: {desig_data.get('unpacked_primary_provisional_designation')}")
    print(f"Packed designation: {desig_data.get('packed_primary_provisional_designation')}")
else:
    print(f"Error: {response.status_code}")

# Extracting Orbital Elements

The `mpc_orb` format contains detailed orbital elements. Here's how to extract the key Keplerian elements.

In [ ]:
# Get orbit for Bennu (OSIRIS-REx target)
response = requests.get(
    "https://data.minorplanetcenter.net/api/get-orb",
    json={"desig": "Bennu"}
)

if response.ok:
    mpc_orb = response.json()[0]['mpc_orb']
    
    # Extract orbital elements
    elements = mpc_orb.get('orbit_data', {}).get('CAR', {}).get('element_data', {})
    epoch_data = mpc_orb.get('epoch_data', {})
    
    print("Orbital Elements for Bennu")
    print("=" * 40)
    print(f"Epoch (MJD): {epoch_data.get('epoch')}")
    print(f"Epoch (TDB): {epoch_data.get('timesystem')}")
    print()
    
    # The elements are in Cartesian format in 'CAR'
    # For Keplerian elements, look in 'COM' (cometary) or extract from the data
    print("Cartesian State Vector:")
    for key, value in elements.items():
        if isinstance(value, (int, float)):
            print(f"  {key}: {value}")
else:
    print(f"Error: {response.status_code}")

# Understanding the mpc_orb Structure

The `mpc_orb` format is hierarchically organized. Let's explore its structure.

In [ ]:
response = requests.get(
    "https://data.minorplanetcenter.net/api/get-orb",
    json={"desig": "Apophis"}
)

if response.ok:
    mpc_orb = response.json()[0]['mpc_orb']
    
    print("Top-level keys in mpc_orb:")
    for key in mpc_orb.keys():
        value = mpc_orb[key]
        if isinstance(value, dict):
            print(f"  {key}: dict with {len(value)} keys")
        elif isinstance(value, list):
            print(f"  {key}: list with {len(value)} items")
        else:
            print(f"  {key}: {type(value).__name__}")
else:
    print(f"Error: {response.status_code}")

# Designation Data

The `designation_data` section contains identifiers and naming information.

In [ ]:
response = requests.get(
    "https://data.minorplanetcenter.net/api/get-orb",
    json={"desig": "Apophis"}
)

if response.ok:
    mpc_orb = response.json()[0]['mpc_orb']
    desig_data = mpc_orb.get('designation_data', {})
    
    print("Designation Data for Apophis:")
    print(json.dumps(desig_data, indent=2))
else:
    print(f"Error: {response.status_code}")

# Physical Parameters

Absolute magnitude (H) and slope parameter (G) are included when available.

In [ ]:
# Compare H magnitudes of different asteroids
asteroids = ["Ceres", "Vesta", "Bennu", "Apophis"]

print("Absolute Magnitudes (H):")
print("-" * 30)

for asteroid in asteroids:
    response = requests.get(
        "https://data.minorplanetcenter.net/api/get-orb",
        json={"desig": asteroid}
    )
    
    if response.ok:
        mpc_orb = response.json()[0]['mpc_orb']
        # H magnitude location may vary in the structure
        h_mag = mpc_orb.get('magnitude_data', {}).get('H')
        if h_mag is None:
            # Try alternative location
            h_mag = mpc_orb.get('H')
        print(f"{asteroid:10s}: H = {h_mag}")
    else:
        print(f"{asteroid:10s}: Error {response.status_code}")

# Error Handling

When querying for an object that doesn't exist, the API returns an error.

In [ ]:
# Query for a non-existent object
response = requests.get(
    "https://data.minorplanetcenter.net/api/get-orb",
    json={"desig": "NotARealAsteroid12345"}
)

print(f"Status code: {response.status_code}")
print(f"Response OK: {response.ok}")

if not response.ok:
    print(f"Error message: Object not found")

# Helper Function

Here's a convenient helper function for querying orbits.

In [ ]:
def get_orbit(designation):
    """
    Retrieve orbital elements for an object from the MPC.
    
    Parameters
    ----------
    designation : str
        Object designation (name, number, or provisional designation)
    
    Returns
    -------
    dict or None
        The mpc_orb dictionary, or None if not found
    """
    response = requests.get(
        "https://data.minorplanetcenter.net/api/get-orb",
        json={"desig": designation}
    )
    
    if not response.ok:
        return None
    
    result = response.json()
    if result and 'mpc_orb' in result[0]:
        return result[0]['mpc_orb']
    return None


def get_orbital_period(designation):
    """
    Calculate the orbital period in years from semi-major axis.
    Uses Kepler's third law: P^2 = a^3 (with P in years, a in AU)
    """
    orbit = get_orbit(designation)
    if orbit is None:
        return None
    
    # Try to find semi-major axis in the orbit data
    orbit_data = orbit.get('orbit_data', {})
    
    # Look for 'a' in various possible locations
    a = None
    for coord_type in ['COM', 'CAR', 'KEP']:
        if coord_type in orbit_data:
            elem_data = orbit_data[coord_type].get('element_data', {})
            if 'a' in elem_data:
                a = elem_data['a']
                break
    
    if a is not None:
        return a ** 1.5  # Kepler's third law
    return None


# Example usage
orbit = get_orbit("Eros")
if orbit:
    print(f"Successfully retrieved orbit for Eros")
    print(f"Designation data: {orbit.get('designation_data', {}).get('iau_name')}")
else:
    print("Failed to retrieve orbit")

# Using the mpc_orb Python Package

The MPC provides a Python package for working with `mpc_orb` format files. You can install it with:

```bash
pip install mpc-orb
```

This package provides validation and parsing utilities for the mpc_orb JSON format.

See: https://github.com/Smithsonian/mpc-public/tree/main/mpc_orb

# Summary

The MPC Orbits API provides access to orbital elements for solar system objects.

Key points:
- **Endpoint**: `https://data.minorplanetcenter.net/api/get-orb`
- **Required parameter**: `desig` - object designation (name, number, or provisional)
- **Response format**: `mpc_orb` JSON format with comprehensive orbital data
- **Current limitation**: Single object queries only

The `mpc_orb` format includes:
- Orbital elements in multiple coordinate systems
- Epoch and time system information
- Designation data (names, numbers, provisional designations)
- Physical parameters (H, G when available)
- Uncertainty information

For questions or feedback, contact the MPC via the [Jira Helpdesk](https://mpc-service.atlassian.net/servicedesk/customer/portal/13/create/148).